#### This notebook preprocesses images for prediction using a DNN trained with Uni-EM.
It opens a series of .tiff files in `path_input` and applies CLAHE (Contrast limited adaptive histogram equalization), which enhances the local contrast of images. It then re-saves them as RGB .png files to `path_results`. After using this notebook on your raw data, the images can be used for segmentation, e.g. with Uni-EM.


In [4]:
%matplotlib notebook
import tifffile as tf
import cv2
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import skimage
from skimage.transform import downscale_local_mean
import tqdm

In [5]:
# user inputs
path_input = r"G:\AG_Morawski\Philip\EM\cc_test_all\test_img - Copy\\"
path_results = r"G:\AG_Morawski\Philip\EM\cc_test_all\test_img - Copy - preprocessed again\\"
downscale_factor = 1 # factor 4 this is the correct factor for 
path_images = [f for f in listdir(path_input) if isfile(join(path_input, f))]

In [8]:
print("Preprocessing Images..")

for i in tqdm.tqdm(range(len(path_images))):
#    test_img = tf.imread(path_input + str(path_images[i])) # use this for tiff
    test_img = cv2.imread(path_input + str(path_images[i]),-1)[:,:,0] # use this for png 
    test_img_clahe = skimage.exposure.equalize_adapthist(test_img,clip_limit=0.01,kernel_size=127)
    test_img_downscaled = downscale_local_mean(test_img_clahe, downscale_factor)
    test_img_rgb_png = cv2.merge((downscale_local_mean(test_img_downscaled,1),  #R
                                downscale_local_mean(test_img_downscaled,1),    #G
                                downscale_local_mean(test_img_downscaled,1)))   #B
    skimage.io.imsave(path_results + path_images[i][:-4] + ".png", (test_img_rgb_png*255).astype('uint8'))


Preprocessing Images..


100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [23:19<00:00, 36.82s/it]


#### To continue:
- Open Uni-EM on the desktop (`Uni-EM\main.exe`) - startup might take a minute
- Open the following three folders by dragging them into the Uni-EM window. There is no feedback to this from the program, only in the accompanying terminal.
    - The folder your preprocessed images are in: `path_results`
    - an empty folder you want your predicted images to be in
    - the model folder `E:\AG_Morawski\Philip\EM\20230419_densenet_12_12_20`
- Click Segmentation -> 2D DNN and then highlight the inference tab
- Select the three folders you already opened above as Image folder, Model Folder, and Output Segmentation folder.
- Select the maximum maximal unit size (2048)
- Click "Execute". The whole process should take a few minutes to an hour, depending on how many images are in the folder.
- After that, continue in `post_uni-em.ipynb`

In [7]:
test_img.shape

(6472, 6210, 3)